# Notebook to develop the model for the project

In [3]:
# All our imports
import torch 
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib as plt
import numpy as np
from random import randint
#for all the plots to be inline
%matplotlib inline 


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
'''Data Set manipulation'''


In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # more analysis required to determine the specifics of the architecture
       
        self.n_output = 8
        self.n_channel = 1
    
        self.layers = nn.Sequential(
#         imageInputLayer([28 28 1])
#         convolution2dLayer(3,8,'Padding','same')
        nn.Conv2d(1, 8, 3), 
#         batchNormalizationLayer
        nn.BatchNorm2d(8),
#         reluLayer
        nn.LeakyReLU(),
#         averagePooling2dLayer(2,'Stride',2)
        nn.MaxPool2d(2, 2),
#         convolution2dLayer(3,16,'Padding','same')
        nn.Conv2d(8, 16, 3),
#         batchNormalizationLayer
        nn.BatchNorm2d(16),
#         reluLayer
        nn.LeakyReLU(),
#         averagePooling2dLayer(2,'Stride',2)
        nn.MaxPool2d(2, 2),
#         convolution2dLayer(3,32,'Padding','same')
        nn.Conv2d(16, 32, 3),
#         batchNormalizationLayer
        nn.BatchNorm2d(32),
#         reluLayer
        nn.LeakyReLU(),
#         convolution2dLayer(3,32,'Padding','same')
        nn.Conv2d(32, 32, 3),
#         batchNormalizationLayer
        nn.BatchNorm2d(32),
#         reluLayer
        nn.LeakyReLU(),
#         Max pooling layer
        nn.MaxPool2d(2, 2)
        )

        self.n_input = 96*96 
        #TODO:actual value might be determined from the computed output of the cnn layers
        # https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

        self.fc = nn.Sequential(
#         fullyConnectedLayer(1)
                nn.Linear(self.n_input, 1024),
                nn.LeakyReLU(),
                nn.Linear(1024, 256),
                nn.LeakyReLU(),
                nn.Linear(256, 64),
                nn.LeakyReLU(),
                nn.Linear(64, self.n_output),
                nn.LeakyReLU()
        )


#         regressionLayer
        self.criterion = nn.MSELoss()       
#         dropoutLayer(0.2)
        self.dropout =  nn.Dropout(p=0.2)
        

        
    def forward(self, x):
        #feedword pass through our network
        x = self.layers(x)
        x = x.view(x.shape[0], -1) #flatten the input tensor
        x = self.fc(x)
        
        return x


    @staticmethod
    def load_checkpoint(new_model, filepath):
        checkpoint = torch.load(filepath)
        model = new_model.Network(checkpoint['input_size'], 
            checkpoint['output_size'],
            checkpoint['hidden_layers']
            )

        model.load_state_dict(checkpoint['state_dict'])

        return model
    
    def save(self):
        self.checkpoint = {
            'input_size': self.n_input, 
            'output_size': self.n_output,
            'hidden_layers': [each.out_features for each in model.hidden_layers],
            'state_dict': model.state_dict()
        }
        torch.save(self.checkpoint, 'checkpoint.pth')
    


In [6]:
    
def validation(model, testloader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in testloader:

        images = images.resize_(images.size()[0], 784)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ## Calculating the accuracy 
        # Model's output is log-softmax, take exponential to get the probabilities
        ps = torch.exp(output)
        # Class with highest probability is our predicted class, compare with true label
        equality = (labels.data == ps.max(1)[1])
        # Accuracy is number of correct predictions divided by all predictions, just take the mean
        accuracy += equality.type_as(torch.FloatTensor()).mean()

    return test_loss, accuracy


def train(model, trainloader, testloader, criterion, optimizer, epochs=5, print_every=40):
    
    steps = 0
    running_loss = 0
    for e in range(epochs):
        # Model in training mode, dropout is on
        model.train()
        for images, labels in trainloader:
            steps += 1
            
            # Flatten images into a 784 long vector
            images.resize_(images.size()[0], 784)
            
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

            if steps % print_every == 0:
                # Model in inference mode, dropout is off
                model.eval()
                
                # Turn off gradients for validation, will speed up inference
                with torch.no_grad():
                    test_loss, accuracy = validation(model, testloader, criterion)
                
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
                
                running_loss = 0
                
                # Make sure dropout and grads are on for training
                model.train()

In [7]:
model = Network()
optimizer = optim.Adam(model.parameters(), lr=0.005)